In [12]:
# Load libraries
import pandas as pd
import sklearn_pandas
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import os # used to create necessary folders
import json
import datetime as dt
import logging
import time
import getpass
import requests
from requests.exceptions import HTTPError
import glob
import sys
from swat import *
import pickle
import zipfile
from math import sqrt
from scipy.stats import kendalltau
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import confusion_matrix                    
from sklearn.metrics import fbeta_score
from scipy.stats import ks_2samp

cas_host = 'sasserver.demo.sas.com'
cas_port= 5570
pd.set_option('display.max_colwidth', -1)
s = CAS(cas_host, cas_port, 'sasdemo', 'Orion123')
s.sessionprop.setsessopt(caslib='Public')


baseurl = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'


NOTE: 'Public' is now the active caslib.


In [13]:
table_name='COVID_FILES'
caslib='Public'


update = False
date = dt.datetime.today()
first_date = dt.datetime.strptime('01-22-2020', '%m-%d-%Y')
df = pd.DataFrame()

while update == False:
    try:
        url = baseurl + str(date.strftime('%m-%d-%Y')) + '.csv'
        r = requests.get(url)
        print('Try: ',url)
        r.raise_for_status()
    except HTTPError:
        date = date + dt.timedelta(days=-1)
    else:
        while date >= first_date:
            url = baseurl + str(date.strftime('%m-%d-%Y')) + '.csv'
            part = pd.read_csv(url, error_bad_lines=False)
            print('Download done:', url)
            df = pd.concat([part,df],sort=True)
            date = date - dt.timedelta(days=1)
            print(df.shape)
        
        if s.tableExists(name=table_name,caslib=caslib).exists:
            s.dropTable(name=table_name,caslib=caslib)
        s.upload_frame(df, importoptions=None, casout={'caslib':caslib, 'name':table_name,'promote':True})
        print('Table updated on server:', url)
        update = True

Try:  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-18-2020.csv
Try:  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-17-2020.csv
Download done: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-17-2020.csv
(276, 8)
Download done: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-16-2020.csv
(548, 8)
Download done: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-15-2020.csv
(806, 8)
Download done: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-14-2020.csv
(1055, 8)
Download done: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03

In [22]:
table=s.CASTable('COVID_FILES').to_frame()
table

,Confirmed,Country,Deaths,Last Update,Latitude,Longitude,Province,Recovered,DATE_UPDATE,Date
0,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Anhui,NaN,2020-01-22 17:00:00,2020-01-22
1,14.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Beijing,NaN,2020-01-22 17:00:00,2020-01-22
2,6.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Chongqing,NaN,2020-01-22 17:00:00,2020-01-22
3,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Fujian,NaN,2020-01-22 17:00:00,2020-01-22
4,NaN,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Gansu,NaN,2020-01-22 17:00:00,2020-01-22
5,26.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Guangdong,NaN,2020-01-22 17:00:00,2020-01-22
6,2.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Guangxi,NaN,2020-01-22 17:00:00,2020-01-22
7,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Guizhou,NaN,2020-01-22 17:00:00,2020-01-22
8,4.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Hainan,NaN,2020-01-22 17:00:00,2020-01-22
9,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Hebei,NaN,2020-01-22 17:00:00,2020-01-22


In [15]:
s.table.save(table=table_name, name=table_name+'.sashdat', replace=True)
s.table.loadTable(path=table_name+'.sashdat',casout={"name":table_name,"caslib":caslib,'replace':True})


NOTE: Cloud Analytic Services saved the file COVID_FILES.sashdat in caslib Public.
NOTE: Cloud Analytic Services made the file COVID_FILES.sashdat available as table COVID_FILES in caslib Public.


[caslib]

 'Public'

[tableName]

 'COVID_FILES'

[casTable]

 CASTable('COVID_FILES', caslib='Public')

+ Elapsed: 0.00098s, user: 0.000719s, sys: 0.000183s, mem: 0.39mb

In [16]:
s.datastep.runcode('data PUBLIC.COVID_FILES;set PUBLIC.COVID_FILES;DATE_UPDATE=input("Last Update"n,ANYDTDTM19.);Date=datepart(DATE_UPDATE);format DATE_UPDATE datetime. Date Date9.;if "Province/State"n="" then "Province/State"n="Country/Region"n; rename "Province/State"n=Province "Country/Region"n=Country;run;')

[InputCasTables]

    casLib         Name  Rows  Columns  \
 0  Public  COVID_FILES  6438  8         
 
                                    casTable  
 0  CASTable('COVID_FILES', caslib='Public')  

[OutputCasTables]

    casLib         Name  Rows  Columns  Append Promoted  \
 0  Public  COVID_FILES  6438  10      NaN      N         
 
                                    casTable  
 0  CASTable('COVID_FILES', caslib='Public')  

+ Elapsed: 0.0756s, user: 0.25s, sys: 0.019s, mem: 8.17mb

In [17]:
s.builtins.loadActionSet("fedSql")    
s.fedSql.execDirect(                                                              #4
    query='''create table COVID_ALL{options replace=true} as select distinct Province,Country,Date,max(deaths) as Deaths,max(Recovered) as Recovered, max(Confirmed) as Confirmed from COVID_FILES group by Country,Province,Date '''
 )

s.fedSql.execDirect(                                                              #4
    query='''create table COVID_GEO{options replace=true} as select distinct Province,Country,avg(Latitude) as lat,avg(Longitude) as long from COVID_FILES  where longitude is not null group by Country,Province'''
 )


s.fedSql.execDirect(                                                              #4
    query='''create table COVID_FINAL{options replace=true} as 
    select distinct t1.Province,t1.Country,t1.Date,t2.lat,t2.long,t1.Deaths,t1.Recovered,t1.Confirmed 
    from COVID_ALL t1 left join COVID_GEO t2 on t1.Country=t2.Country and t1.Province=t2.Province'''
 )


NOTE: Added action set 'fedSql'.
NOTE: CASDAL driver. Creation of a DATE column has been requested, but is not supported by the CASDAL driver. A DOUBLE PRECISION column will be created instead. A DATE format will be associated with the column.
NOTE: CASDAL driver. Creation of a DATE column has been requested, but is not supported by the CASDAL driver. A DOUBLE PRECISION column will be created instead. A DATE format will be associated with the column.
NOTE: Table COVID_ALL was created in caslib Public with 3965 rows returned.
NOTE: Table COVID_GEO was created in caslib Public with 473 rows returned.
NOTE: CASDAL driver. Creation of a DATE column has been requested, but is not supported by the CASDAL driver. A DOUBLE PRECISION column will be created instead. A DATE format will be associated with the column.
NOTE: CASDAL driver. Creation of a DATE column has been requested, but is not supported by the CASDAL driver. A DOUBLE PRECISION column will be created instead. A DATE format will be 

+ Elapsed: 0.105s, user: 0.131s, sys: 0.0614s, mem: 40.7mb

In [18]:
table_name='COVID_FINAL'
LIB_OUT='PUBLIC'
s.table.save(table=table_name, name=table_name+'.sashdat', replace=True)
s.table.loadTable(path=table_name+'.sashdat',casout={"name":table_name,"caslib":LIB_OUT,'replace':True})

NOTE: Cloud Analytic Services saved the file COVID_FINAL.sashdat in caslib Public.
NOTE: Cloud Analytic Services made the file COVID_FINAL.sashdat available as table COVID_FINAL in caslib PUBLIC.


[caslib]

 'PUBLIC'

[tableName]

 'COVID_FINAL'

[casTable]

 CASTable('COVID_FINAL', caslib='PUBLIC')

+ Elapsed: 0.000859s, user: 0.000577s, sys: 0.000237s, mem: 0.742mb

In [33]:
table.columns = table.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
table


,confirmed,country,deaths,last_update,latitude,longitude,province,recovered,date_update,date
0,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Anhui,NaN,2020-01-22 17:00:00,2020-01-22
1,14.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Beijing,NaN,2020-01-22 17:00:00,2020-01-22
2,6.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Chongqing,NaN,2020-01-22 17:00:00,2020-01-22
3,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Fujian,NaN,2020-01-22 17:00:00,2020-01-22
4,NaN,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Gansu,NaN,2020-01-22 17:00:00,2020-01-22
5,26.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Guangdong,NaN,2020-01-22 17:00:00,2020-01-22
6,2.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Guangxi,NaN,2020-01-22 17:00:00,2020-01-22
7,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Guizhou,NaN,2020-01-22 17:00:00,2020-01-22
8,4.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Hainan,NaN,2020-01-22 17:00:00,2020-01-22
9,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Hebei,NaN,2020-01-22 17:00:00,2020-01-22
